In [1]:
import pickle

import pandas as pd

In [28]:
with open("./data/df_compare.pkl", "rb") as f:
    df_compare = pickle.load(f)

In [29]:
df_compare_original = df_compare.copy()

In [30]:
# 'original'列の辞書を新しい列に展開
df_compare[["original_label", "original_sentiment"]] = df_compare["original"].apply(
    pd.Series
)
# 'predicted'列の辞書を新しい列に展開
df_compare[["predicted_label", "predicted_sentiment"]] = df_compare["predicted"].apply(
    pd.Series
)
# 'original'列と'predicted'列を削除
df_compare.drop(columns=["original", "predicted"], inplace=True)

In [31]:
df_compare = df_compare[["original_label", "original_sentiment", "predicted_label", "predicted_sentiment", "text"]]

In [36]:
# ラベルとセンチメントが一致しているかどうかを判定

df_compare["is_match_label"] = df_compare["original_label"] == df_compare["predicted_label"]
df_compare["is_match_sentiment"] = df_compare["original_sentiment"] == df_compare["predicted_sentiment"]
df_compare["is_match_all"] = df_compare["is_match_label"] & df_compare["is_match_sentiment"]

num_match_label = df_compare["is_match_label"].sum()
num_match_sentiment = df_compare["is_match_sentiment"].sum()
num_match_all = df_compare["is_match_all"].sum()
num_all = len(df_compare)

print(f"num_all: {num_all}")
print(f"num_match_label: {num_match_label}")
print(f"num_match_sentiment: {num_match_sentiment}")
print(f"num_match_all: {num_match_all}")

num_all: 682
num_match_label: 486
num_match_sentiment: 634
num_match_all: 459


In [35]:
df_compare

,original_label,original_sentiment,predicted_label,predicted_sentiment,text,is_match_label,is_match_sentiment,is_match_all
4,移動・交通,満足,移動・交通,満足,鉄道路線が充実している。どこにいくにも便利。,True,True,True
8,自然景観,満足,公共空間,満足,海も近く県立公園もあり、リラックスできるので。,False,True,False
12,移動・交通,満足,移動・交通,満足,今居住している武蔵小杉はJR横須賀線、南武線、東急東横線、目黒線と選択の余地がたくさんあり、...,True,True,True
13,買物・飲食,満足,買物・飲食,満足,スーパーマーケットもたくさんあり、選択の余地が広い,True,True,True
14,遊び・娯楽,不満,遊び・娯楽,不満,かなり発展した街なのに映画館がない。ライブハウスがない。,True,True,True
...,...,...,...,...,...,...,...,...
1941,デジタル生活,不満,地域行政,不満,行政関係の申請や交付をするときに、市役所まで出向いて行う必要があり、デジタル行政関係が遅れて...,False,True,False
1942,医療・福祉,不満,医療・福祉,不満,高齢者福祉・支援や障害者支援などを包括的に対応できる具体的な体制や組織が不十分であると感じている。,True,True,True
1945,医療・福祉,不満,医療・福祉,不満,南部地区に総合病院がないので設置してほしい。,True,True,True
1946,買物・飲食,不満,買物・飲食,不満,藤沢駅周辺のショッピング環境を更に改善してほしい。,True,True,True


In [39]:
# 一致していないデータを確認
df_compare[~df_compare["is_match_label"]]

,original_label,original_sentiment,predicted_label,predicted_sentiment,text,is_match_label,is_match_sentiment,is_match_all
8,自然景観,満足,公共空間,満足,海も近く県立公園もあり、リラックスできるので。,False,True,False
15,自然災害,不満,医療・福祉,不満,台風である地域が浸水した際、エレベーターが止まったりと都市ならではの被害があった。##近くを...,False,True,False
16,自然災害,満足,公共空間,満足,横須賀市は神奈川県の中でも温暖なエリアで、例えば隣の横浜市などが雪の為、交通障害を起きている...,False,True,False
55,事故・犯罪,不満,公共空間,不満,病院があるのに時々暴走族が大きい音でバイク運転をしている事。,False,True,False
57,事故・犯罪,満足,地域行政,満足,よくわからないけど、たぶん、米軍基地があっても沖縄程問題がおこらないので,False,True,False
...,...,...,...,...,...,...,...,...
1917,地域行政,不満,デジタル生活,不満,デジタルの活用でもっと便利にして欲しい。,False,True,False
1922,買物・飲食,不満,事故・犯罪,不満,駅周辺、夜になると飲み屋のキャッチなど治安がよくない,False,True,False
1925,移動・交通,不満,地域行政,不満,七夕祭りのときに道路を封鎖するため混雑するところ,False,True,False
1941,デジタル生活,不満,地域行政,不満,行政関係の申請や交付をするときに、市役所まで出向いて行う必要があり、デジタル行政関係が遅れて...,False,True,False


In [41]:
df_compare[~df_compare["is_match_sentiment"]]

,original_label,original_sentiment,predicted_label,predicted_sentiment,text,is_match_label,is_match_sentiment,is_match_all
255,地域行政,不満,その他,不明,特に改善点はありませんが、一つは選択しなければならなかったので。,False,False,False
259,地域行政,不満,デジタル生活,不明,土日や休日も開館、対応して頂きたい。その代わりに、特定の平日を休みにして頂いても結構。,False,False,False
274,事故・犯罪,不満,犯罪・事故,満足,比較的、犯罪などは少ないし、警察のパトロールも行っているので、安心はしているが、それでも犯罪...,False,False,False
320,自然の恵み,満足,自然景観,不明,川や山が近くにあるけれど、不便なほど田舎ではないところ。,False,False,False
327,医療・福祉,不満,医療・福祉,不明,これから老人になるので近隣に必要な病院が配備されると助かる,True,False,False
338,移動・交通,不満,移動・交通,満足,広い自治体なのでバスだけではなく路面電車が出来ると更に暮らしやすくなるだろう。##,True,False,False
340,自然災害,満足,環境共生,不明,この地域に住んでから、洪水、崖崩れなどには遭遇していない。,False,False,False
347,都市景観,不満,都市景観,不明,ｍｍ21でビルが乱立しているように感じる,True,False,False
367,地域行政,不満,地域行政,満足,選挙の投票などはスーパーなどでできる様にして欲しい。,True,False,False
370,遊び・娯楽,不満,遊び・娯楽,満足,たくさん娯楽施設が欲しいとは思わないが、景観のいい場所や、アウトレットモールなどがあればいい...,True,False,False
